In [1]:
import pandas as pd
import numpy as np

In [18]:
df = pd.read_csv('product_data_status.csv')
df.head()

,Main Reference,Product Name,Brand,Attributes,Product Reference,Product Model,Sale Price,Regular Price,RRP Price,Categories,...,Product Specific Images,On Sale,Active,Discontinued,Meta Title,Meta Keywords,Meta Description,Weight,Friendly URL,Image Location
0,1405,Leather Shirt Jacket Birmingham Black/ant,Ashwood,Size: XS,140502,DM-SHIRT,99.0,189.0,0.0,"&middot;&nbsp;Mens,Leather Jackets,Offers,Sale...",...,5,Y,Y,N,Y,Y,Y,0.0,https://www.leathercompany.co.uk/mens-c4/leath...,https://www.leathercompany.co.uk/images/produc...
1,1405,Leather Shirt Jacket Birmingham Black/ant,Ashwood,Size: S,140503,DM-SHIRT,99.0,189.0,0.0,"&middot;&nbsp;Mens,Leather Jackets,Offers,Sale...",...,5,Y,Y,N,Y,Y,Y,0.0,https://www.leathercompany.co.uk/mens-c4/leath...,https://www.leathercompany.co.uk/images/produc...
2,1405,Leather Shirt Jacket Birmingham Black/ant,Ashwood,Size: M,140504,DM-SHIRT,99.0,189.0,0.0,"&middot;&nbsp;Mens,Leather Jackets,Offers,Sale...",...,5,Y,Y,N,Y,Y,Y,0.0,https://www.leathercompany.co.uk/mens-c4/leath...,https://www.leathercompany.co.uk/images/produc...
3,1405,Leather Shirt Jacket Birmingham Black/ant,Ashwood,Size: L,140505,DM-SHIRT,99.0,189.0,0.0,"&middot;&nbsp;Mens,Leather Jackets,Offers,Sale...",...,5,Y,Y,N,Y,Y,Y,0.0,https://www.leathercompany.co.uk/mens-c4/leath...,https://www.leathercompany.co.uk/images/produc...
4,1405,Leather Shirt Jacket Birmingham Black/ant,Ashwood,Size: XL,140506,DM-SHIRT,99.0,189.0,0.0,"&middot;&nbsp;Mens,Leather Jackets,Offers,Sale...",...,5,Y,Y,N,Y,Y,Y,0.0,https://www.leathercompany.co.uk/mens-c4/leath...,https://www.leathercompany.co.uk/images/produc...


In [22]:
df.shape

(6900, 25)

In [19]:
df['FinalCategory'] = df['Categories'].str.split(',').str[-1].str.replace('<br />', '').str.replace('<b', '')
len(df['FinalCategory'].value_counts())

59

In [26]:
df[df['Product Name'].str.contains("Super Bomber")]

,Main Reference,Product Name,Brand,Attributes,Product Reference,Product Model,Sale Price,Regular Price,RRP Price,Categories,...,On Sale,Active,Discontinued,Meta Title,Meta Keywords,Meta Description,Weight,Friendly URL,Image Location,FinalCategory


In [28]:
len(df['Product Name'].unique())

1500